# Single SQL Agent coding

In [23]:
#feel free to put it in your model of choice
llm_config={"model": "gpt-4-turbo"}

In [24]:
import pandas as pd

In [25]:
# create a config.py file to store your openai key
from config import openai_key
llm_config = {"model": "gpt-4o-mini", "api_key": openai_key}

In [26]:
from autogen.coding import CodeBlock
from autogen.coding.jupyter import JupyterCodeExecutor, LocalJupyterServer

with LocalJupyterServer() as server:
    executor = JupyterCodeExecutor(server)
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code="print('Hello, World!')"),
            ]
        )
    )

exit_code=0 output='Hello, World!\n' output_files=[]


In [27]:
import duckdb

In [28]:
schema = duckdb.sql("DESCRIBE SELECT * FROM 'yellow_tripdata_2024-01.parquet'")

In [29]:
schema

┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type │  null   │   key   │ default │  extra  │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ VendorID              │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ tpep_pickup_datetime  │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ tpep_dropoff_datetime │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ passenger_count       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ trip_distance         │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ RatecodeID            │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ store_and_fwd_flag    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ PULocationID          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ DOLocationID          │ INTEGER     │ 

In [30]:
from autogen import ConversableAgent, AssistantAgent

In [31]:
message = f"Here is the schema of the parquet file yellow_tripdata_2024-01.parquet: {schema}. Create a SQL code to get the whatever the user requests and return the results with the associated function.."

In [32]:
code_sql_agent_system_message =  "You are a a helpful assistant that generates Postgres SQL code based on user request. Make sure you query from the parquet file, such as DailyRevenue.parquet instead of DailyRevenue in the SQL query as this is duckdb." + message


code_sql_agent_system_message += """You have access to the following user defined functions. They can be accessed from the module called `functions` by their function names.

    For example, if there was a function called `foo` you could import it by writing `from functions import foo`.
    Here is the function you have access to:
    def duckdb_query(query):
        import duckdb
        result = duckdb.sql('(query)).to_df()
        print(result)
    """


In [34]:
code_sql_agent = AssistantAgent(
    name="code_sql_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    system_message=code_sql_agent_system_message
)

[autogen.oai.client: 09-18 01:34:48] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [35]:
def duckdb_query(query):
    import duckdb
    result = duckdb.sql(f"{query}").to_df()
    print(result)
    

In [36]:
server = LocalJupyterServer()



code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={
        "executor": JupyterCodeExecutor(server, output_dir="coding"),
    },
    human_input_mode="ALWAYS",
    default_auto_reply=
    "Please continue. If everything is done, reply 'TERMINATE'.",
)

In [37]:
code_sql_agent_system_message

"You are a a helpful assistant that generates Postgres SQL code based on user request. Make sure you query from the parquet file, such as DailyRevenue.parquet instead of DailyRevenue in the SQL query as this is duckdb.Here is the schema of the parquet file yellow_tripdata_2024-01.parquet: ┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐\n│      column_name      │ column_type │  null   │   key   │ default │  extra  │\n│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │\n├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤\n│ VendorID              │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │\n│ tpep_pickup_datetime  │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │\n│ tpep_dropoff_datetime │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │\n│ passenger_count       │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │\n│ trip_distance         │ DOUBLE      │ YES     │ NULL    │ NU

In [ ]:
#Input your query in the message section

chat_result = code_executor_agent.initiate_chat(
    code_sql_agent,
    message="""
    Please generate me the average pickup time for a daily basis for the parquet file yellow_tripdata_2024-01.parquet for DuckDB.?
    
    Here is an example query that works for DuckDB based on the following question:

    {Please generate me the average number of rides a day on split by weekday basis for the parquet file yellow_tripdata_2024-01.parquet for DuckDB. Can you split it by each day of the week?}

    Here was the query:

    {SELECT 
        EXTRACT(DOW FROM tpep_pickup_datetime) AS weekday,
        COUNT(*)::FLOAT / COUNT(DISTINCT CAST(tpep_pickup_datetime AS DATE)) AS average_rides
    FROM 
        read_parquet('yellow_tripdata_2024-01.parquet')
    GROUP BY 
        weekday
    ORDER BY 
        weekday;
    }

    Make sure to use the duckdb_query function to execute the query.
    
     """

)

# Group Chat

In [11]:
task = """

Please select the appropriate metrics for a GTM strategy and then breakdown the new york taxi market data for January 2024 based on the parquet file yellow_tripdata_2024-01.parquet with SQL. Then create a GTM strategy blog postfor how to break in to the market as a ride sharing app.

Here is an example query that works for DuckDB based on the following question:

{Please generate me the average number of rides a day on split by weekday basis for the parquet file yellow_tripdata_2024-01.parquet for DuckDB. Can you split it by each day of the week?}

Here was the query:

{SELECT 
    EXTRACT(DOW FROM tpep_pickup_datetime) AS weekday,
    COUNT(*)::FLOAT / COUNT(DISTINCT CAST(tpep_pickup_datetime AS DATE)) AS average_rides
FROM 
    read_parquet('yellow_tripdata_2024-01.parquet')
GROUP BY 
    weekday
ORDER BY 
    weekday;
}

Make sure to use the duckdb_query function to execute the query.

"""

In [12]:
import autogen

In [13]:
server = LocalJupyterServer()

user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

[autogen.oai.client: 09-18 01:20:11] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [14]:
code_planner_agent_system_message = message + """You have access to the following user defined functions. They can be accessed from the module called `functions` by their function names.

    
    Here is the function that you should use:
    
    def duckdb_query(query):
        import duckdb
        result = duckdb.sql((query)).to_df()
        print(result)
    """

In [15]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    "Python code with SQL. Please only suggest information that can be "
    "retrieved using Python code with SQL. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround. Here is the schema of the database: {schema}",

    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

[autogen.oai.client: 09-18 01:20:14] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [16]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message = code_planner_agent_system_message,
    description="An engineer that writes Python and/or SQL code and then function on the plan and schema provided by the planner.",
)

[autogen.oai.client: 09-18 01:20:15] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [17]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "executor": JupyterCodeExecutor(server, output_dir="coding"),
    },
)

In [18]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

[autogen.oai.client: 09-18 01:20:16] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [19]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [20]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


[autogen.oai.client: 09-18 01:20:18] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [21]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=20,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [22]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

[autogen.oai.client: 09-18 01:20:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.
Admin (to chat_manager):



Please select the appropriate metrics for a GTM strategy and then breakdown the new york taxi market data for January 2024 based on the parquet file yellow_tripdata_2024-01.parquet with SQL. Then create a GTM strategy blog postfor how to break in to the market as a ride sharing app.

Here is an example query that works for DuckDB based on the following question:

{Please generate me the average number of rides a day on split by weekday basis for the parquet file yellow_tripdata_2024-01.parquet for DuckDB. Can you split it by each day of the week?}

Here was the query:

{SELECT 
    EXTRACT(DOW FROM tpep_pickup_datetime) AS weekday,
    COUNT(*)::FLOAT / COUNT(DISTINCT CAST(tpep_pickup_datetime AS DATE)) AS average_rides
FROM 
    read_parquet('yellow_tripdata_2024-01.parquet')
GROUP BY 
    weekday
ORDER BY 
   

[autogen.oai.client: 09-18 01:20:21] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.

Next speaker: Engineer

Engineer (to chat_manager):

Here’s a SQL query to breakdown the New York taxi market data for January 2024. This example will extract the total number of rides, average fare amount, and total revenue (sum of fare amount) per day. Then, I will provide you with a GTM strategy blog post for entering the ride-sharing market. 

```python
# Define the SQL query to get metrics for the New York taxi market
query = """
SELECT 
    CAST(tpep_pickup_datetime AS DATE) AS ride_date,
    COUNT(*) AS total_rides,
    AVG(fare_amount) AS average_fare,
    SUM(total_amount) AS total_revenue
FROM 
    read_parquet('yellow_tripdata_2024-01.parquet')
GROUP BY 
    ride_date
ORDER BY 
    ride_date;
"""

# Execute the query using the duckdb_query function
duckdb_query(query)
```

### GTM Strategy Blog Post: Riding into the New York Ta